In [1]:
import time

import numpy as np
import pandas as pd

## Stochastic FSM
aka [Probabilistic Automaton](https://en.wikipedia.org/wiki/Probabilistic_automaton)

In [2]:
states = pd.DataFrame({'state_id': ['off', 'rinse', 'ready_espresso', 'serve_espresso'],'output': ['', 'dirty, hot water', '', 'espresso']}).set_index('state_id')

In [3]:
transitions = pd.DataFrame(columns=['state_id', 'event', 'transition_to'])

new_entry = {'state_id':'off',
             'event':'press_powerOn',
             'transition_to':'rinse'}
transitions = transitions.append(new_entry, ignore_index=True)

new_entry = {'state_id':'rinse',
             'event':'t',
             'transition_to':'ready_espresso'}
transitions = transitions.append(new_entry, ignore_index=True)


new_entry = {'state_id':'ready_espresso',
             'event':'press_serve',
             'transition_to':['serve_espresso', 'rinse'],
             'transition_prob': [0.9, 0.1]}

transitions = transitions.append(new_entry, ignore_index=True)

new_entry = {'state_id':'ready_espresso',
             'event':'press_powerOff',
             'transition_to':'off'}
transitions = transitions.append(new_entry, ignore_index=True)

new_entry = {'state_id':'serve_espresso',
             'event':'t',
             'transition_to':'ready_espresso'}

transitions = transitions.append(new_entry, ignore_index=True)

transitions

,state_id,event,transition_to,transition_prob
0,off,press_powerOn,rinse,NaN
1,rinse,t,ready_espresso,NaN
2,ready_espresso,press_serve,"[serve_espresso, rinse]","[0.9, 0.1]"
3,ready_espresso,press_powerOff,off,NaN
4,serve_espresso,t,ready_espresso,NaN


In [4]:
states = states.infer_objects()
transitions = transitions.infer_objects()

## Run FSM

In [5]:
def setKeyBindings(transitions):

    k = {}
    for e in transitions['event']:
        if e == 't':
            continue
        else:
            k[input('Type key to bind with event ' + e + ': ')] = e

    return k

In [6]:
state0 = None
keyBindings = {'1': 'press_powerOn', 's': 'press_serve', '0': 'press_powerOff'}
nRepeats = 10

In [7]:
def runFSM(states, transitions, state0=None, keyBindings=None, nRepeats=10, sleepTime=5):

    state0 = transitions.loc[0, 'state_id'] if state0 is None else state0
    keyBindings = setKeyBindings(transitions) if keyBindings is None else keyBindings

    current_state = state0

    for iTral in range(nRepeats):
        print('\n\t' + states.loc[current_state, 'output'])
        time.sleep(.2)
        if 't' in transitions.loc[transitions['state_id']==current_state, 'event'].values:
            time.sleep(sleepTime)
            event = 't'
        else:
            event = keyBindings[input('waiting for input: ')]

        next_state = transitions.loc[(transitions['state_id']==current_state)*(transitions['event']==event),
                                     'transition_to'].squeeze()
        if isinstance(next_state, list):
            candidate_states = transitions.loc[(transitions['state_id']==current_state)*(transitions['event']==event),
                                     'transition_to'].squeeze()
            p = transitions.loc[(transitions['state_id']==current_state)*(transitions['event']==event),
                                                 'transition_prob'].squeeze()
            
            next_state = np.random.choice(candidate_states, p=p).item()
            
        current_state = next_state
        
    return
    

In [8]:
runFSM(states, transitions, keyBindings=keyBindings, sleepTime=.5)


	
waiting for input: 1

	dirty, hot water

	
waiting for input: s

	espresso

	
waiting for input: s

	espresso

	
waiting for input: s

	espresso

	
waiting for input: s

	dirty, hot water


<img src=https://media.giphy.com/media/YQM4N3OWpe18y7hDQA/giphy.gif width=400>